In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

import os
import json
from tqdm import tqdm

In [3]:
from customResNet import setup_custom_resnet
from MLP import create_mlp_model
from leNet import create_lenet
from alexNet import create_AlexNet
from VGG import create_vggcustommodel
from cnnModel import create_cnn_model
from myCNN import create_my_cnn_model

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 32x32
    transforms.RandomHorizontalFlip(),  # Randomly flip the images horizontally
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # Randomly apply affine transformations: shearing and scaling
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the tensors
])


# Load the dataset
dataset = datasets.ImageFolder('/Users/kapil/.keras/datasets/flower_photos', transform = transform)

# Splitting the dataset
length_of_dataset = len(dataset)
total_indices = list(range(length_of_dataset))
first_split = int(np.floor(0.6 * length_of_dataset))  # 60% of the data for training
second_split = int(np.floor(0.8 * length_of_dataset))  # Additional 20% for validation, total 80%

np.random.seed(40)
np.random.shuffle(total_indices)

train_indices = total_indices[:first_split]
validation_indices = total_indices[first_split:second_split]
test_indices = total_indices[second_split:]

train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=validation_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=test_sampler)

In [5]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [6]:
def train_model(model, criterion, optimizer, number_of_epochs):
    # Initialize lists to monitor training and validation loss
    train_losses = []
    validation_losses = []
    
    for epoch in range(1, number_of_epochs + 1):
        # Training phase
        model.train()
        train_loss = 0.0
        # Wrap the train_loader with tqdm for a progress bar
        train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch}/{number_of_epochs} [Training]')
        for data, target in train_loader_tqdm:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)  # Multiply by batch size to accumulate total loss
            train_loader_tqdm.set_postfix(loss=loss.item())  # Optionally update the progress bar with the current loss
        
        # Validation phase
        model.eval()
        validation_loss = 0.0
        # Wrap the validation_loader with tqdm for a progress bar
        validation_loader_tqdm = tqdm(validation_loader, desc=f'Epoch {epoch}/{number_of_epochs} [Validation]')
        with torch.no_grad():
            for data, target in validation_loader_tqdm:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                validation_loss += loss.item() * data.size(0)  # Multiply by batch size to accumulate total loss
                validation_loader_tqdm.set_postfix(loss=loss.item())  # Optionally update the progress bar with the current loss
        
        # Calculate average losses
        train_loss = train_loss / len(train_loader.dataset)  # Divide by total number of samples
        validation_loss = validation_loss / len(validation_loader.dataset)  # Divide by total number of samples
        
        # Append losses to lists for each epoch
        train_losses.append(train_loss)
        validation_losses.append(validation_loss)
        
        # Print training and validation statistics
        print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {validation_loss:.6f}')
    
    # Return the lists of losses
    return [train_losses[-1], validation_losses[-1]]

In [7]:
def test_model(model, criterion, model_name, results, loss):
    train_loss, validation_loss = loss[0], loss[1]
    test_loss = 0.0
    class_correct = list(0. for i in range(5))
    class_total = list(0. for i in range(5))

    model.eval() # Prepare model for evaluation

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item() * data.size(0)
        _, pred = torch.max(output, 1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))

        # Calculate correct predictions for each class
        for i in range(len(target)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # Calculate and print avg test loss
    classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

    # Calculate and print avg test loss
    test_loss = test_loss / len(test_loader.dataset)
    # print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(5):  # Adjusted to iterate through all 5 classes
        if class_total[i] > 0:
            print(f"Test Accuracy of {classes[i]}: {100 * class_correct[i] / class_total[i]:2.0f}% ({np.sum(class_correct[i]):2.0f}/{np.sum(class_total[i]):2.0f})")

        else:
            print(f"Test Accuracy of {classes[i]:>10}: N/A (no training examples)")

    overall_accuracy = 100. * np.sum(class_correct) / np.sum(class_total)

    # Update the results dictionary correctly for all five classes
    results[model_name] = {
        'daisy_accuracy': 100 * class_correct[0] / class_total[0] if class_total[0] > 0 else 0,
        'dandelion_accuracy': 100 * class_correct[1] / class_total[1] if class_total[1] > 0 else 0,
        'roses_accuracy': 100 * class_correct[2] / class_total[2] if class_total[2] > 0 else 0,
        'sunflowers_accuracy': 100 * class_correct[3] / class_total[3] if class_total[3] > 0 else 0,
        'tulips_accuracy': 100 * class_correct[4] / class_total[4] if class_total[4] > 0 else 0,
        'overall_accuracy': overall_accuracy,
        'test_loss': test_loss,
        'train_loss': train_loss,
        'validation_loss': validation_loss
    }

    # print(f"Results for CNN_Model1")
    print(f"Overall Accuracy: {overall_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.6f}")

In [8]:
def save_results_to_file(model_name, results, results_file='test_results.txt'):
    # Check if the results file already exists
    if os.path.isfile(results_file):
        # File exists, load the existing data and update it
        with open(results_file, 'r') as file:
            existing_results = json.load(file)
    else:
        # File does not exist, create a new dictionary to hold the results
        existing_results = {}

    # Update the existing results with the new results
    existing_results[model_name] = results[model_name]

    # Save the updated results back to the file
    with open(results_file, 'w') as file:
        json.dump(existing_results, file, indent=4)

    print(f"Results for {model_name} have been updated in {results_file}.")

In [9]:
results = {}

In [30]:
# Training CNN model 1
number_of_classes = 5
model = create_cnn_model(number_of_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
number_of_epochs = 25

cnn_model_loss = train_model(model, criterion, optimizer, number_of_epochs)

CNN_model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=5, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


Epoch 1/25 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.05it/s, loss=1.24]



Epoch: 1 	Training Loss: 0.828718 	Validation Loss: 0.224907


Epoch 2/25 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.85it/s, loss=1.07]



Epoch: 2 	Training Loss: 0.636479 	Validation Loss: 0.208974


...


Epoch: 23 	Training Loss: 0.159698 	Validation Loss: 0.193560


Epoch 24/25 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.17it/s, loss=0.96]



Epoch: 24 	Training Loss: 0.145057 	Validation Loss: 0.182931


Epoch 25/25 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.62it/s, loss=1.09]


Epoch: 25 	Training Loss: 0.127569 	Validation Loss: 0.191690


In [31]:
# Testing CNN model 1
test_model(model, criterion, "cnn_model", results, cnn_model_loss)
save_results_to_file("cnn_model", results)

Test Accuracy of daisy: 81% (104/128)
Test Accuracy of dandelion: 73% (122/168)
Test Accuracy of roses: 56% (85/151)
Test Accuracy of sunflowers: 80% (103/129)
Test Accuracy of tulips: 62% (98/158)
Overall Accuracy: 69.75%
Test Loss: 0.231416
Results for cnn_model have been updated in test_results.txt.


In [10]:
# training Multi-layer perceptron
number_of_classes = 5  # Adjust based on your dataset
img_shape = (3, 224, 224)  # Adjust based on your dataset

model = create_mlp_model(number_of_classes, img_shape)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
number_of_epochs = 30

mlp_loss = train_model(model, criterion, optimizer, number_of_epochs)

MLP(
  (input_fc): Linear(in_features=150528, out_features=250, bias=True)
  (hidden_fc): Linear(in_features=250, out_features=100, bias=True)
  (output_fc): Linear(in_features=100, out_features=5, bias=True)
)
Trainable parameters: 37657855


Epoch 1/30 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.19it/s, loss=2.42]



Epoch: 1 	Training Loss: 2.418767 	Validation Loss: 0.452588


Epoch 2/30 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.19it/s, loss=1.92]



Epoch: 2 	Training Loss: 1.134153 	Validation Loss: 0.436200


...


Epoch: 28 	Training Loss: 0.502102 	Validation Loss: 0.279940


Epoch 29/30 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.03it/s, loss=1.47]



Epoch: 29 	Training Loss: 0.487948 	Validation Loss: 0.277854


Epoch 30/30 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.16it/s, loss=1.35]


Epoch: 30 	Training Loss: 0.469940 	Validation Loss: 0.291919


In [11]:
# testing MLP
test_model(model, criterion, "MLP", results, mlp_loss)
save_results_to_file("MLP", results)

Test Accuracy of daisy: 41% (52/128)
Test Accuracy of dandelion: 44% (74/168)
Test Accuracy of roses: 34% (52/151)
Test Accuracy of sunflowers: 67% (86/129)
Test Accuracy of tulips: 51% (81/158)
Overall Accuracy: 47.00%
Test Loss: 0.298969
Results for MLP have been updated in test_results.txt.


In [12]:
# training lenet
number_of_classes = 5  # Adjust based on your dataset
# img_shape = (3, 32, 32)  # Adjust based on your dataset

model = create_lenet(number_of_classes)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
number_of_epochs = 20

lenet_loss = train_model(model, criterion, optimizer, number_of_epochs)

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=44944, out_features=120, bias=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_3): Linear(in_features=84, out_features=5, bias=True)
)
The model has 5,406,861 trainable parameters


Epoch 1/20 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.28it/s, loss=0.888]



Epoch: 1 	Training Loss: 0.805000 	Validation Loss: 0.227615


Epoch 2/20 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.56it/s, loss=1.21]



Epoch: 2 	Training Loss: 0.685440 	Validation Loss: 0.217559


...


Epoch: 18 	Training Loss: 0.277799 	Validation Loss: 0.196550


Epoch 19/20 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.04it/s, loss=1.49]



Epoch: 19 	Training Loss: 0.239429 	Validation Loss: 0.222164


Epoch 20/20 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.19it/s, loss=1.19]


Epoch: 20 	Training Loss: 0.243388 	Validation Loss: 0.195976


In [13]:
# testing lenet
test_model(model, criterion, "lenet", results, lenet_loss)
save_results_to_file("lenet", results)

Test Accuracy of daisy: 66% (84/128)
Test Accuracy of dandelion: 72% (121/168)
Test Accuracy of roses: 40% (60/151)
Test Accuracy of sunflowers: 73% (94/129)
Test Accuracy of tulips: 66% (104/158)
Overall Accuracy: 63.08%
Test Loss: 0.243279
Results for lenet have been updated in test_results.txt.


In [17]:
# training alexnet
number_of_classes = 5  # Adjust based on your dataset
img_shape = (3, 224, 224)  # Adjust based on your dataset

model = create_AlexNet(number_of_classes)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
number_of_epochs = 60

alexnet_loss = train_model(model, criterion, optimizer, number_of_epochs)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, 

Epoch 1/60 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.48it/s, loss=1.32]



Epoch: 1 	Training Loss: 0.935288 	Validation Loss: 0.268482


Epoch 2/60 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.35it/s, loss=1.13]



Epoch: 2 	Training Loss: 0.798988 	Validation Loss: 0.258848


...


Epoch: 58 	Training Loss: 0.257960 	Validation Loss: 0.203389


Epoch 59/60 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.27it/s, loss=0.82]



Epoch: 59 	Training Loss: 0.254330 	Validation Loss: 0.168992


Epoch 60/60 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.22it/s, loss=0.58]


Epoch: 60 	Training Loss: 0.243102 	Validation Loss: 0.164100


In [18]:
# testing alexnet
test_model(model, criterion, "alexnet", results, alexnet_loss)
save_results_to_file("alexnet", results)

Test Accuracy of daisy: 66% (84/128)
Test Accuracy of dandelion: 76% (127/168)
Test Accuracy of roses: 60% (90/151)
Test Accuracy of sunflowers: 76% (98/129)
Test Accuracy of tulips: 73% (116/158)
Overall Accuracy: 70.16%
Test Loss: 0.197039
Results for alexnet have been updated in test_results.txt.


In [32]:
# Training myCNN model
number_of_classes = 5
model = create_my_cnn_model(number_of_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
number_of_epochs = 35

my_cnn_model_loss = train_model(model, criterion, optimizer, number_of_epochs)

my_CNN_model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.4, inplace=False)
  (fc1): Linear(in_features=12544, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=5, bias=True)
)


Epoch 1/35 [Validation]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.24it/s, loss=1.6]



Epoch: 1 	Training Loss: 0.968729 	Validation Loss: 0.319845


Epoch 2/35 [Validation]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.35it/s, loss=1.38]



Epoch: 2 	Training Loss: 0.886495 	Validation Loss: 0.277751


...


Epoch: 33 	Training Loss: 0.259307 	Validation Loss: 0.137365


Epoch 34/35 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.58it/s, loss=0.69]



Epoch: 34 	Training Loss: 0.266126 	Validation Loss: 0.143655


Epoch 35/35 [Validation]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.42it/s, loss=1.13]


Epoch: 35 	Training Loss: 0.252257 	Validation Loss: 0.142843


In [33]:
# Testing myCNN model
test_model(model, criterion, "my_cnn_model", results, my_cnn_model_loss)
save_results_to_file("my_cnn_model", results)

Test Accuracy of daisy: 70% (89/128)
Test Accuracy of dandelion: 88% (147/168)
Test Accuracy of roses: 42% (64/151)
Test Accuracy of sunflowers: 78% (101/129)
Test Accuracy of tulips: 76% (120/158)
Overall Accuracy: 70.98%
Test Loss: 0.169096
Results for my_cnn_model have been updated in test_results.txt.
